In [1]:
import LeakDetector
import json
import glob
from os.path import basename
import pandas as pd
from urllib.parse import urlparse
# from parallel_pandas import ParallelPandas
from pandarallel import pandarallel
from tld import get_fld

In [7]:
# folder_path = "../leak-detector/2023-10-17_inner_collector/*.json"  # replace with the path to your folder
folder_path = "2023-10-11_inner_collector/*.json"  # replace with the path to your folder
# folder_path = "../leak-detector/2023-10-11_inner_collector/osheaga.com_5288.json"  # replace with the path to your folder
# folder_path = "../leak-detector/2023-10-11_inner_collector/an.gov.br_ea4d.json"
# folder_path = '../leak-detector/2023-10-11_inner_collector/versatel.de_b202.json'
# folder_path = '../leak-detector/2023-10-11_inner_collector/postman-echo.com_0eb5.json'



# ParallelPandas.initialize(n_cpu=8, split_factor=4, disable_pr_bar=False)
pandarallel.initialize(nb_workers=8, progress_bar=True)

extracted_list = []

for json_path in glob.glob(folder_path):
    json_name = basename(json_path)

    try:
        with open(json_path, encoding='utf-8-sig') as file:
            results = json.load(file)
    except Exception as e:
        print("ERROR: Cannot load the json", json_name, e)
        continue
    
    try:
        init_url = results["initialUrl"]
        final_url = results["finalUrl"]
    except Exception as e:
        print("ERROR: Cannot find the url", json_name, e)
        continue

    for req in results ["data"]["requests"]:
        req_url = req["url"]
        req_method = req.get("method")
        req_data = req.get("postData")
        ref_data = req.get('requestHeaders', {}).get('referer', '')
        # if req_data is not None:
        #     req_data = req_data.replace("\\/", "/").encode().decode('unicode_escape', 'surrogatepass')

        extracted_details = (init_url, final_url, req_url, req_method, ref_data, req_data)
        extracted_list.append(extracted_details)

        

extractedDF = pd.DataFrame(extracted_list, columns=["init_url", "final_url", "req_url", "req_method", "ref_data", "req_data"])
# extractedDF.to_csv("leak_first.csv", index=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
ERROR: Cannot find the url urlChecker_inner.json 'initialUrl'
ERROR: Cannot find the url metadata.json 'initialUrl'


In [8]:
extractedDF


,init_url,final_url,req_url,req_method,ref_data,req_data
0,http://wheatonarts.org/,https://www.wheatonarts.org/,http://wheatonarts.org/,GET,,
1,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/,GET,,
2,http://wheatonarts.org/,https://www.wheatonarts.org/,https://fonts.googleapis.com/css?family=Open+S...,GET,https://www.wheatonarts.org/,
3,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,
4,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,
...,...,...,...,...,...,...
79825,http://osheaga.com/,https://osheaga.com/fr,https://www.gstatic.com/eureka/clank/93/cast_s...,GET,https://www.youtube.com/,
79826,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=38692092893660...,GET,https://osheaga.com/,
79827,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",..."
79828,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",..."


In [ ]:
extractedDF[extractedDF['req_method'] == 'POST']

In [4]:
def selection_for_search(path_search,params_search,query_search):
    save = []
    var = {'path_search': path_search, 'params_search': params_search, 'query_search': query_search}
    for key, value in var.items():
        if value is not None and value != '':
            save.append(value)
    return save
        
A = 'asc'
B = None
C = ''

selection_for_search(A,B,C)

['asc']

In [5]:
import tldextract

def leaky(row):
 
    final_url = row['final_url']
    req_url = row['req_url']
    req_data = row["req_data"]
    ref_data = row["ref_data"]
    
    netloc_search = urlparse(final_url).netloc
    hostname_search = urlparse(final_url).hostname
    try:
        path_search = urlparse(final_url).path
        if path_search == '/':
            path_search = ""
        params_search = urlparse(final_url).params
        query_search = urlparse(final_url).query
        selected = selection_for_search(path_search,params_search,query_search)
    # print (selected)
    except Exception as e:
        print("ERROR: Cannot find the url", json_name, e)
        

    # print (netloc_search, hostname_search, path_search, params_search, query_search)

    search_terms = [final_url, netloc_search, hostname_search] + selected

    # try:
    leak_detector = LeakDetector.LeakDetector(
                search_terms, encoding_set=LeakDetector.ENCODINGS_NO_ROT,
                    hash_set=LeakDetector.LIKELY_HASHES,
                    encoding_layers=3,
                    hash_layers=3,
                    debugging=False
                )
    # except Exception as e:
    #     print("ERROR: Cannot do leak detector", json_name, e)
    #     return None, None


    url_leaks = None
    post_leaks = None
    referer_leaks = None
    try:
        valid_url = get_fld(final_url)
    except Exception as e:
        print("ERROR: ", final_url, e)
        return None, None, None        
    for __ in range(1):
        if (req_url.startswith('blob:') or req_url.startswith('chromewebdata') or req_url.startswith('chrome-extension:') or req_url.startswith('localhost')):
            continue
        try:
            if not tldextract.extract(req_url).suffix:
                continue
        except Exception as e:
            print("ERROR: ", req_url, e)
            continue
        if get_fld(req_url) != valid_url:
            try:
                url_leaks = leak_detector.check_url(req_url, encoding_layers=3) 
            except Exception as err:
                # print("ERROR: Cannot do url leak detector", req_url, json_name, err)
                pass
            try:
                post_leaks = leak_detector.check_post_data(req_data, encoding_layers=3)
            except Exception as err:
                # print("ERROR: Cannot do post leak detector", req_data, json_name, err)
                pass
            try:
                referer_leaks = leak_detector.check_url(ref_data, encoding_layers=3)
            except Exception as err:
                pass
        else:
            url_leaks = []
            referer_leaks = []
            try:
                post_leaks = leak_detector.check_post_data(req_data, encoding_layers=3)
            except Exception as err:
                # print("ERROR: Cannot do post leak detector", req_data, json_name, err)
                pass




    return str(url_leaks), str(post_leaks), str(referer_leaks)
    # return str(post_leaks)
    # return str(url_leaks)


extractedDF[['url_leak','post_leak','referer_leak']] = extractedDF.parallel_apply(leaky, axis=1, result_type='expand')
# extractedDF[['urlLeak','postLeak','referer_leaks']] = extractedDF.apply(leaky, axis=1, result_type='expand')
# extractedDF['postLeak'] = extractedDF.parallel_apply(leaky, axis=1, result_type='expand')
# extractedDF['urlLeak'] = extractedDF.parallel_apply(leaky, axis=1, result_type='expand')

Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse url  empty separator
Cannot parse u

In [6]:
extractedDF

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks
0,http://wheatonarts.org/,https://www.wheatonarts.org/,http://wheatonarts.org/,GET,,,[],[],[]
1,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/,GET,,,[],[],[]
2,http://wheatonarts.org/,https://www.wheatonarts.org/,https://fonts.googleapis.com/css?family=Open+S...,GET,https://www.wheatonarts.org/,,[],[],[]
3,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,,[],[],[]
4,http://wheatonarts.org/,https://www.wheatonarts.org/,https://www.wheatonarts.org/wp-content/themes/...,GET,https://www.wheatonarts.org/,,[],[],[]
...,...,...,...,...,...,...,...,...,...
79825,http://osheaga.com/,https://osheaga.com/fr,https://www.gstatic.com/eureka/clank/93/cast_s...,GET,https://www.youtube.com/,,[],[],[]
79826,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=38692092893660...,GET,https://osheaga.com/,,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79827,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79828,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],[],[]


In [37]:
refLeak = extractedDF[(extractedDF['referer_leaks'] != '[]') & (extractedDF['referer_leaks'] != 'None')]
refLeak = refLeak[refLeak['referer_leaks'].notnull()]

In [38]:
refLeak

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks
208,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/webworke...,GET,https://www.google.com/recaptcha/api2/anchor?a...,,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
210,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/reload?k...,POST,https://www.google.com/recaptcha/api2/anchor?a...,\nlLirU0na9roYU3wDDisGJEVTù\n03AFcWeA4mfv4J0...,[],[],"[('urlencode', 'base64', 'www.virtuaoperator.p..."
480,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://googleads.g.doubleclick.net/pagead/adv...,GET,https://googleads.g.doubleclick.net/pagead/ads...,,[],[],"[('www.farsondigitalwatercams.com',), ('urlenc..."
848,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/tab/r...,GET,https://www.facebook.com/v14.0/plugins/page.ph...,,"[('www.clackesd.org',), ('urlencode', 'https:/...",[],"[('www.clackesd.org',), ('urlencode', 'www.cla..."
849,http://clackesd.k12.or.us/,https://www.clackesd.org/,https://www.facebook.com/platform/plugin/page/...,POST,https://www.facebook.com/v14.0/plugins/page.ph...,event_name=page_plugin.tab.configured&page_id=...,[],"[('www.clackesd.org',), ('urlencode', 'https:/...","[('www.clackesd.org',), ('urlencode', 'www.cla..."
...,...,...,...,...,...,...,...,...,...
74822,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/polyfills.js,GET,https://app.grow.me/register?callback=https%3A...,,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
74825,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/main.js,GET,https://app.grow.me/register?callback=https%3A...,,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
74827,http://veganinthefreezer.com/,https://veganinthefreezer.com/,https://app.grow.me/static/js/app.6.61.0.js,GET,https://app.grow.me/register?callback=https%3A...,,[],[],"[('urlencode', 'https://veganinthefreezer.com/..."
77427,http://headstuff.org/,https://headstuff.org/,https://gum.criteo.com/sid/json?origin=publish...,GET,https://gum.criteo.com/syncframe?origin=publis...,,"[('headstuff.org',), ('urlencode', 'headstuff....",[],"[('headstuff.org',), ('urlencode', 'headstuff...."


In [89]:
ref_leak_print =  refLeak.drop(['urlLeak', 'postLeak', 'req_data'], axis=1)
ref_leak_print.to_csv("leak_ref_print.csv", index=False)

In [46]:
refLeak.req_method.value_counts()

GET        441
POST       123
OPTIONS     11
PUT          5
Name: req_method, dtype: int64

In [79]:
refLeak[refLeak['req_url'].str.contains('meta')]

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks


In [45]:
extractedDF.to_csv("leak2.csv", index=False)

In [ ]:
extractedDFPartial = extractedDF.drop("req_data", axis=1)

In [ ]:
extractedDFPartial

In [ ]:
maskUrl = (extractedDF['urlLeak'] == '[]') | (extractedDF['urlLeak'] == 'None')
df1 = extractedDF[~maskUrl]
df1.drop("postLeak", axis=1, inplace=True)
df1.drop("req_data", axis=1, inplace=True)
df1.to_csv("leak_url.csv", index=False)

In [48]:
urlLeak = extractedDF[(extractedDF['urlLeak'] != '[]') & (extractedDF['urlLeak'] != 'None')]
urlLeak = urlLeak[urlLeak['urlLeak'].notnull()]
urlLeak

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks
122,http://wheatonarts.org/,https://www.wheatonarts.org/,https://analytics.google.com/g/collect?v=2&tid...,POST,https://www.wheatonarts.org/,None,"[('www.wheatonarts.org',), ('urlencode', 'http...",None,[]
201,http://virtuaoperator.pl/,https://www.virtuaoperator.pl/,https://www.google.com/recaptcha/api2/anchor?a...,GET,https://www.virtuaoperator.pl/,,"[('urlencode', 'base64', 'www.virtuaoperator.p...",[],[]
331,http://retailgators.com/,https://www.retailgators.com/,https://xbytedev.co/xbyte-analytics/matomo.php...,POST,https://www.retailgators.com/,None,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
336,http://retailgators.com/,https://www.retailgators.com/,wss://vts.zohopublic.com/watchws?x-e=embed1.xb...,None,,None,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
341,http://retailgators.com/,https://www.retailgators.com/,https://www.google-analytics.com/j/collect?v=1...,POST,https://www.retailgators.com/,None,"[('urlencode', 'https://www.retailgators.com/'...",None,[]
...,...,...,...,...,...,...,...,...,...
79811,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=16057439283596...,GET,https://osheaga.com/,,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79812,http://osheaga.com/,https://osheaga.com/fr,https://adservice.google.com/ddm/fls/z/dc_pre=...,GET,https://osheaga.com/,,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]
79818,http://osheaga.com/,https://osheaga.com/fr,https://connect.facebook.net/signals/config/38...,GET,https://osheaga.com/,,"[('urlencode', 'osheaga.com'), ('osheaga.com',)]",[],[]
79819,http://osheaga.com/,https://osheaga.com/fr,https://www.facebook.com/tr/?id=19916247543847...,GET,https://osheaga.com/,,"[('urlencode', '/fr'), ('urlencode', 'https://...",[],[]


In [85]:
url_print = urlLeak.drop(['postLeak', 'referer_leaks', 'ref_data', 'req_data'], axis=1)
url_print.to_csv("leak_url_print.csv", index=False)

In [54]:
urlLeak.req_method.value_counts()

GET        3825
POST       1107
OPTIONS      47
PUT           1
Name: req_method, dtype: int64

In [84]:
urlLeak[urlLeak['req_url'].str.contains('google')].count()

init_url         1736
final_url        1736
req_url          1736
req_method       1736
ref_data         1736
req_data          962
urlLeak          1736
postLeak         1736
referer_leaks    1736
dtype: int64

In [56]:
postLeak = extractedDF[(extractedDF['postLeak'] != '[]') & (extractedDF['postLeak'] != 'None')]
postLeak = postLeak[postLeak['postLeak'].notnull()]
postLeak

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks
431,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://maps.googleapis.com/$rpc/google.intern...,POST,https://www.farsondigitalwatercams.com/,"[[[43.25051468911406,-25.557122316294674],[63....",[],"[('www.farsondigitalwatercams.com',), ('https:...",[]
533,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://dc.services.visualstudio.com/v2/track,POST,https://www.farsondigitalwatercams.com/,"[{""time"":""2023-10-11T18:46:16.869Z"",""iKey"":""78...",[],"[('www.farsondigitalwatercams.com',), ('urlenc...",[]
557,http://farsondigitalwatercams.com/,https://www.farsondigitalwatercams.com/,https://bam.nr-data.net/events/1/9ac7216156?a=...,POST,https://www.farsondigitalwatercams.com/,"bel.7;1,c,,6bh,540,d6,'initialPageLoad,'https:...","[('www.farsondigitalwatercams.com',), ('urlenc...","[('www.farsondigitalwatercams.com',), ('https:...",[]
624,http://limonhost.net/,https://www.limonhost.net/,wss://socket.tidio.co/socket.io/?ppk=xezog98i6...,None,,"420[""visitorRegister"",{""id"":""a412a907db674f33a...",[],"[('www.limonhost.net',), ('https://www.limonho...",[]
632,http://limonhost.net/,https://www.limonhost.net/,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""20fb6f55-5bcf-4fb9-9805-f6ed7f975dda"",...",[],"[('www.limonhost.net',), ('https://www.limonho...",[]
...,...,...,...,...,...,...,...,...,...
79815,http://osheaga.com/,https://osheaga.com/fr,https://analytics.tiktok.com/api/v2/pixel,POST,https://osheaga.com/,"{""event"":""Pageview"",""message_id"":""messageId-16...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79820,http://osheaga.com/,https://osheaga.com/fr,https://analytics.tiktok.com/api/v2/pixel/act,POST,https://osheaga.com/,"{""message_id"":""messageId-1697048461638-8852546...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]
79821,http://osheaga.com/,https://osheaga.com/fr,https://www.youtube.com/youtubei/v1/log_event?...,POST,https://www.youtube.com/embed/Ku2hXbXvjrA,"{""context"":{""client"":{""hl"":""en"",""gl"":""US"",""cli...",[],"[('osheaga.com',)]",[]
79827,http://osheaga.com/,https://osheaga.com/fr,wss://ws.hotjar.com/api/v2/client/ws?v=4,None,,"{""sid"":""6fe607cf-45ed-46ba-8956-f8be89ef6d7c"",...",[],"[('https://osheaga.com/fr',), ('/fr',), ('oshe...",[]


In [86]:
leak_post_print = postLeak.drop(['urlLeak', 'referer_leaks', 'ref_data'], axis=1)
leak_post_print.to_csv("leak_post_print.csv", index=False)

In [61]:
postLeak.req_method.value_counts()

POST    1304
PUT        1
Name: req_method, dtype: int64

In [72]:
postLeak[postLeak['req_url'].str.contains('meta')]

,init_url,final_url,req_url,req_method,ref_data,req_data,urlLeak,postLeak,referer_leaks


In [ ]:
maskBlank = extractedDF['postLeak'] == '[]'
maskNone = extractedDF['postLeak'] == 'None'
dfBlank = extractedDF[~maskBlank]
dfBlank


In [ ]:
df2 = dfBlank[~maskNone]
df2.drop("urlLeak", axis=1, inplace=True)
# df2.drop("req_url", axis=1, inplace=True)
df2.to_csv("leak_post.csv", index=False)
df2

In [ ]:
count_list_elements = lambda x: len(x) if isinstance(x, list) else 0

# extractedDF['urlLeakCount'] = extractedDF['urlLeak'].apply(count_list_elements)
extractedDF['postLeakCount'] = extractedDF['postLeak'].apply(count_list_elements)


In [ ]:
def count_list(x):
    if x == '[]' or x == None:
        return 0
    else:
        return 1
    
extractedDF['postLeakCount'] = extractedDF['postLeak'].apply(count_list)


In [ ]:
extractedDF

In [ ]:
extractedDF[extractedDF['postLeakCount'] > 0]

In [ ]:
count_url_leak = extractedDF['urlLeakCount'].sum()
count_url_leak

In [ ]:
count_post_leak = extractedDF['postLeakCount'].sum()
count_post_leak